In [1]:
import os, pickle, sys
import matplotlib.pyplot as plt
from scipy import stats
import numpy as np
import glob
from prettytable import PrettyTable
from tqdm import tqdm

In [3]:
t=None
all_ds = {}
all_acc = {}
allc = {}
all_metrics = {}
all_runs = {}
metric_names = ['grad_norm', 'snip', 'grasp', 'fisher', 'synflow', 'jacob_cov']
for fname,rname in [('../results_release/nasbench2/nb2_cf10_seed42_dlrandom_dlinfo1_initwnone_initbnone.p','CIFAR10')]:
    runs=[]
    f = open(fname,'rb')
    while(1):
        try:
            runs.append(pickle.load(f))
        except EOFError:
            break
    f.close()
    print(fname, len(runs))
    print(runs[0])
    
    all_runs[fname]=runs
    all_ds[fname] = {}
    metrics={}
    for k in metric_names:
        metrics[k] = []
    acc = []
    
    if t is None:
        hl=['Dataset']
        hl.extend(metric_names)
        t = PrettyTable(hl)
    
    for r in runs:
        for k,v in r['logmeasures'].items():
            if k in metrics:
                metrics[k].append(v)
        acc.append(r['testacc'])
        
    all_ds[fname]['metrics'] = metrics
    all_ds[fname]['acc'] = acc

    res = []
    crs = {}
    for k in hl:
        if k=='Dataset':
            continue
        v = metrics[k]
        cr = abs(stats.spearmanr(acc,v,nan_policy='omit').correlation)
        #print(f'{k} = {cr}')
        res.append(round(cr,3))
        crs[k]=cr
    
    ds = rname
    all_acc[ds]=acc
    allc[ds]=crs
    t.add_row([ds]+res)
      
    all_metrics[ds] = metrics
print(t)

../results_release/nasbench2/nb2_cf10_seed42_dlrandom_dlinfo1_initwnone_initbnone.p 15625
{'i': 0, 'arch': '|avg_pool_3x3~0|+|nor_conv_1x1~0|skip_connect~1|+|nor_conv_1x1~0|skip_connect~1|skip_connect~2|', 'logmeasures': {'grad_norm': 3.2280678749084473, 'snip': 5.514030933380127, 'grasp': -0.1634197235107422, 'fisher': 0.004466889891773462, 'jacob_cov': -151.3307806940146, 'plain': 0.028314922004938126, 'synflow_bn': 16.958625670825334, 'synflow': 1.9405668633541168e+17}, 'trainacc': 88.91866665852865, 'valacc': 81.98266665690103, 'testacc': 81.52}
+---------+-----------+-------+-------+--------+---------+-----------+
| Dataset | grad_norm |  snip | grasp | fisher | synflow | jacob_cov |
+---------+-----------+-------+-------+--------+---------+-----------+
| CIFAR10 |   0.594   | 0.596 | 0.514 |  0.36  |  0.737  |   0.731   |
+---------+-----------+-------+-------+--------+---------+-----------+


In [8]:
print("origin_cr:",crs['synflow'])

origin_cr: 0.7367344450455346
